In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np
import copy
import random

In [2]:
ctx = utils.try_all_gpus()
net0 = MLP()
net0.initialize(init=mx.init.Xavier(),ctx=ctx)
x = nd.random.uniform(0,255,shape=(1,28,28),ctx=ctx[0])
net0(x)
net1 = MLP()
net1.initialize(init=mx.init.Xavier(),ctx=ctx)
net1(x)
net = MLP()
net.initialize(init=mx.init.Xavier(),ctx=ctx)

In [3]:
def zero_net(net,ctx):
    net.initialize(mx.init.Xavier(),ctx=ctx)
    x = nd.random.uniform(0,255,shape=(1,28,28),ctx=ctx[0])
    net(x)
    for layer in net:
        try:
            layer.weight.data()[:] = nd.zeros(shape=layer.weight.data().shape,ctx=ctx[0])[:]
            layer.bias.data()[:] = nd.zeros(shape=layer.bias.data().shape,ctx=ctx[0])[:]
        except:
            pass
def merge(net, net_a, net_b):
    id = 0
    for layer in net:
        layer.weight.data()[:] = net_a[id].weight.data()[:] + net_b[id].weight.data()[:]
        layer.weight.data()[:] /= 2
        id += 1

In [4]:
zero_net(net,ctx)

In [5]:
merge(net, net0, net1)

In [10]:
net0[2].weight.data()[0]


[-0.04882678  0.23818293 -0.24821572 -0.16126832  0.10961187  0.03712472
  0.03792918  0.20792401 -0.13360946  0.00510776  0.01323965  0.23732153
 -0.23124877  0.23984703  0.04325113 -0.23741527  0.24448195 -0.1265881
 -0.10332404 -0.2794188   0.09533933  0.19496202 -0.2096892   0.08381489
  0.1231972   0.19441763 -0.11993213 -0.13398495 -0.18042088 -0.05819055
  0.04926866  0.03008154 -0.27329624 -0.19081466  0.18732962 -0.0741436
 -0.28207335 -0.20134957  0.10126558  0.03964731 -0.13097927  0.1160273
  0.13394177 -0.12046157  0.26321396 -0.0379921  -0.14308377  0.1458514
  0.0433712  -0.05917148  0.0524174   0.2255418   0.04114708  0.07911482
 -0.15770356  0.22298822  0.25783822  0.1025407  -0.03011182 -0.02893162
  0.19727793  0.27254364  0.1136024  -0.218571  ]
<NDArray 64 @gpu(0)>

In [11]:
net1[2].weight.data()[0]


[ 0.1364229  -0.02727365 -0.00543365  0.26779732 -0.15523595  0.10281926
 -0.13989271 -0.23617202 -0.25170007 -0.25261745 -0.03734939 -0.00692636
 -0.10718127  0.21698013  0.11181659  0.27130982 -0.0696197   0.06700557
 -0.18246403  0.02420282 -0.27069297  0.20195061 -0.24644908  0.13886252
  0.10216323 -0.01218927 -0.02636939  0.10084704  0.02083164  0.0609616
  0.22590223  0.1222688   0.27924547 -0.01737121 -0.16122569 -0.02504945
  0.09287217  0.23145297 -0.13478667 -0.20660108 -0.27298674 -0.15420818
  0.14714527  0.21731094 -0.10249929  0.23031792 -0.0663667   0.08302358
  0.05029613 -0.09984213  0.18853036  0.01122543  0.07345447 -0.28471586
  0.2122226  -0.10714461 -0.12896663 -0.04245496  0.16973612  0.21944776
 -0.17902869  0.10244039  0.25786248 -0.02498388]
<NDArray 64 @gpu(0)>

In [12]:
net[2].weight.data()[0]


[ 0.04379806  0.10545464 -0.12682468  0.0532645  -0.02281204  0.06997199
 -0.05098177 -0.01412401 -0.19265476 -0.12375484 -0.01205487  0.11519758
 -0.16921502  0.22841358  0.07753386  0.01694728  0.08743113 -0.02979127
 -0.14289403 -0.12760799 -0.08767682  0.19845632 -0.22806914  0.1113387
  0.11268021  0.09111418 -0.07315076 -0.01656896 -0.07979462  0.00138552
  0.13758545  0.07617517  0.00297461 -0.10409293  0.01305196 -0.04959653
 -0.09460059  0.0150517  -0.01676054 -0.08347689 -0.201983   -0.01909044
  0.14054352  0.04842468  0.08035734  0.09616291 -0.10472523  0.11443749
  0.04683366 -0.07950681  0.12047388  0.11838362  0.05730078 -0.10280052
  0.02725952  0.0579218   0.06443579  0.03004287  0.06981215  0.09525807
  0.00912462  0.18749201  0.18573244 -0.12177745]
<NDArray 64 @gpu(0)>